In [2]:
import extract
import pandas as pd
import os
import geopandas as gpd

In [3]:
import importlib
importlib.reload(extract)

<module 'extract' from 'd:\\Users\\User\\Downloads\\RA Work\\acs\\extract.py'>

In [11]:
# open state_fips
state_fips = pd.read_csv('state_fips.csv')

In [10]:
# for every folder in raw_tract_data
for folder in os.listdir('raw_tract_data'):
    # split folder into list by underscore
    folder_tokens = folder.split('_')
    
    # get state column from state_fips where code is third item in list as int
    state = state_fips[state_fips['code'] == int(folder_tokens[2])]['state'].values[0].lower()

    # if last item in folder is 'tract' year is 2000, otherwise it is 20 + last two digits
    year = '2020' if folder_tokens[-1] == 'tract' else '20' + folder_tokens[-1][-2:]

    # if tract file already exists, skip
    if os.path.exists(f'tract_polygons/{state}_tracts_{year}.csv'):
        continue

    # get the filepath of the shp file in the folder with the same name
    shp_file = os.path.join('raw_tract_data', folder, folder + '.shp')

    tract_geos = extract.read_tracts(shp_file)
    
    # save as acs\tract_polygons\berkeley_tracts_2010.csv
    tract_geos.to_csv(f'tract_polygons/{state}_tracts_{year}.csv', index=False)

In [46]:
# for alaska upper (weird KML)
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
districts = gpd.read_file('raw_geo_data\\alaska_upper_2022.kml', driver='KML')
# rename Name to district
districts = districts.rename(columns={'Name': 'district'})
# only keep district and geometry
districts = districts[['district', 'geometry']]
# only keep the part of the district field in the form 0200A
districts['district'] = districts['district'].str.extract(r'(\d{4}[A-Z])')
# only keep the last letter
districts['district'] = districts['district'].str[-1]
# save to geo_polygons folder as alaska_upper_districts_2022.csv
# districts.to_csv('geo_polygons/alaska_upper_districts_2022.csv', index=False)

In [6]:
# for SHP
# read in shp
districts = gpd.read_file(r'raw_geo_data\Takoma Park Wards\Wards.shp')
districts

,ID,WARD,Area,Area_acre,geometry
0,1,WARD 1,1.241621e+07,285.036880,"POLYGON ((1308455.448 476329.557, 1308871.437 ..."
1,2,WARD 2,1.218601e+07,279.752227,"POLYGON ((1313784.759 480988.747, 1313645.375 ..."
2,3,WARD 3,1.160095e+07,266.321108,"POLYGON ((1308871.568 475921.040, 1309222.861 ..."
3,4,WARD 4,4.759064e+06,109.253071,"POLYGON ((1309056.321 481544.047, 1308233.986 ..."
4,5,WARD 5,6.590607e+06,151.299507,"POLYGON ((1313168.469 481037.974, 1313092.950 ..."
5,6,WARD 6,9.955564e+06,228.548305,"POLYGON ((1313833.542 480920.399, 1313844.711 ..."


In [7]:
# only keep district and geometry
district_name = 'ID'
year=2022
desc='takoma_park_wards'
districts = districts[[district_name, 'geometry']]
# rename to district and convert to int
districts = districts.rename(columns={district_name: 'district'})
districts['district'] = districts['district'].astype(int)

In [8]:
districts

,district,geometry
0,1,"POLYGON ((1308455.448 476329.557, 1308871.437 ..."
1,2,"POLYGON ((1313784.759 480988.747, 1313645.375 ..."
2,3,"POLYGON ((1308871.568 475921.040, 1309222.861 ..."
3,4,"POLYGON ((1309056.321 481544.047, 1308233.986 ..."
4,5,"POLYGON ((1313168.469 481037.974, 1313092.950 ..."
5,6,"POLYGON ((1313833.542 480920.399, 1313844.711 ..."


In [9]:
# save to geo_polygons folder
districts.to_csv(f'geo_polygons/{desc}_{year}.csv', index=False)